In [14]:
import numpy as np
import pandas as pd
import os
from dotenv import load_dotenv
from datetime import timedelta, datetime
import time
from tqdm import tqdm


from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now
from tinkoff.invest.schemas import InstrumentStatus, Quotation
from tinkoff.invest.constants import INVEST_GRPC_API

In [2]:
# Load secrets

load_dotenv("../secrets.env")

token_ro = os.getenv("TINKOFF_TOKEN_RO")

In [34]:
years_needed = 3

def quot_to_price(quot: Quotation):
    return float(f"{quot.units}.{quot.nano}")

imoex_shares_path = "../data/tickers_info/imoex_shares.csv"
imoex_index_shares_path = "../data/tickers_info/imoex_index_shares.csv"
imoex_index_tickers_path = "../data/tickers_info/imoex_index_tickers.csv"
figi_load_path = "../data/historical/figi_to_load.txt"
daily_data_folder = f"../data/historical/daily_data_for_{years_needed}_years_till_{datetime.today().strftime('%Y-%m-%d')}/"

def daily_data_fname(ticker, uid):
    return f"{ticker}_{uid}.csv"

In [35]:
_client = Client(token_ro, target=INVEST_GRPC_API)
client = _client.__enter__()

In [36]:
shares = pd.read_csv(imoex_index_shares_path)
shares.head(5)

,figi,ticker,name,uid
0,BBG000RMWQD4,ENPG,En+ Group,e2bd2eba-75de-4127-b39c-2f2dbe3866c3
1,BBG004PYF2N3,POLY,Polymetal,127361c2-32ec-448c-b3ec-602166f537ea
2,BBG004RVFCY3,MGNT,Магнит,ca845f68-6c43-44bc-b584-330d2a1e5eb7
3,BBG00ZHCX1X2,FIXP,Fix Price Group,ba64a3c7-dd1d-4f19-8758-94aac17d971b
4,BBG00178PGX3,VKCO,VK,3c0748ce-9b49-43e9-b788-048a6cb65174


In [37]:
!mkdir $daily_data_folder

In [38]:
for idx, row in tqdm(shares.iterrows()):
    share_data = {}
    to_s = [now() - timedelta(days=365 * i) for i in range(0, years_needed)]
    for to in to_s:
        candles = None
        while candles is None:
            try:
                candles = client.market_data.get_candles(from_=to - timedelta(days=365),
                                                         to=to,
                                                         interval=CandleInterval.CANDLE_INTERVAL_DAY,
                                                         instrument_id=row["uid"]).candles
            except:
                print("Too many requests. Sleeping for 5 seconds.")
                time.sleep(5)

        for candle in candles:
            share_data[candle.time.date()] = [quot_to_price(candle.open),
                                              quot_to_price(candle.close)]
    share_df = pd.DataFrame.from_dict(share_data, orient="index", columns=["open", "close"]).sort_index()
    share_df.to_csv(daily_data_folder + daily_data_fname(row["ticker"], row["uid"]))


40it [00:04,  8.23it/s]


In [39]:
_client.__exit__(None, None, None)

False